In [ ]:
# Import necessary libraries and modules
%matplotlib widget
import numpy as np
import scipy.constants as scp
import math
import matplotlib.pyplot as plt
import Photon_Class as P  # Photon_Class contains the definition of the Photon class
import Sim_data as data # Sim_data contains definiations for QE and abs data
import random
import csv
import MonteCarlo_V2 as MC  # MonteCarlo_V2 contains the definition of the start_sim function

np.random.seed(0)

# ------------------------------------- Inputs and Simulation Setup: -----------------------------------------

# WLS PLATE CONSTANTS
w, h, l = 0.05, 0.015, 0.05  # Dimensions for WLS block
n_wls = 1.58  # Refractive index of WLS plate
n_air = 1.0003  # Refractive index of water
ca = math.asin(n_air/n_wls)  # Critical angle between WLS and water
bx1, bx2 = 0, w  # x boundaries
by1, by2 = 0, h  # y boundaries
bz1, bz2 = 0, l  # z boundaries 
boundaries = np.array([[0, w], [0, h], [0, l]])  # Boundaries
atl = 0.006  # Attenuation length

# PMT CONSTANTS
dist = 0.1 #distance between WLS block and PMT
cx, cy, cz = w/2, h+dist, l/2  # PMT center positions
rx, ry, rz = 0.001, 0, 0.001  # PMT radii
thetaPMT = np.linspace(0, 2*np.pi, 100)
pmt_center = np.array([cx, cy, cz])
pmt_radii = np.array([rx, ry, rz])

# PHOTON CONSTANTS
v_mag_wls = scp.c * n_wls  # Velocity magnitude in wls
v_mag_air = scp.c * n_air  # Velocity magnitude in air

# SIMULATION CONSTANTS
nruns = 1000

# Create a header list for the output CSV file
header_list = ['Run', 'Initial_wl_nm', 'Absorption_length_m', 'Absorption_coordinate_m', 
               'Isotropic_emmission_angle_theta_deg', 'Isotropic_emmission_angle_phi_deg', 'Shifted_wl_nm',
               'Refraction_coordinate_m', 'Hit_coordinate_m', 'Detected',
               'Num_reflections', 'Quantumn_efficiency', 'X_positions_m', 
               'Y_positions_m', 'Z_positions_m', 'Time_s', 'Timed_out']
path = 'outputs/7mm_30cm/'
out_file = path + 'sim_output0.txt'

with open(out_file, 'w', newline='') as file:
    writer = csv.writer(file,  delimiter=';')
    writer.writerow(header_list)

    # Run the simulation for each run
    for n in range(nruns):
        print("- LOG -")
        print("Run:", n)

        # Refresh these variables every run
        x, y, z = w/4, h, l/4  # x, y, and z origin positions
        orig_pos = np.array([x, y, z])  # Position vector
        theta, phi = 280 * (math.pi/180), 90 * (math.pi/180)  # Angle of refraction into the WLS plate
        vx, vy, vz = v_mag_wls * math.sin(theta) * math.cos(phi), v_mag_wls * math.sin(theta) * math.sin(phi), v_mag_wls * math.cos(theta)  # Velocity origin directions 
        velocity = np.array([vx, vy, vz])  # Velocity vector
        init_wl = 380  # Cherenkov wavelength (nm)

        # Run the simulation using the start_sim function from MonteCarlo_V2
        (absl, abs_coords, theta_iso, phi_iso, shwl, refract_coords, hit_coords, detected, 
         reflects, pmtqe, xpos, ypos, zpos, t, timeout) = MC.start_sim(orig_pos, velocity, init_wl, atl, 
                                                                      boundaries, pmt_center, pmt_radii, ca)

        # Write the results of the simulation to the CSV file
        writer.writerow([n, init_wl , absl , abs_coords, theta_iso, phi_iso, shwl, refract_coords, 
                         hit_coords, detected, reflects, pmtqe, xpos, ypos, zpos, t, timeout])
        

'''
Remove reflective boundary
Read out the refraction points from simulation
Determine what muons have hit the pmt at some distance

'''